In [1]:
import numpy as np
import scipy as sp
import pandas as pd

import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

/Users/zybell/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Task 1

In [2]:
NPI = pd.read_table('Medicare_Physician_and_Other_Supplier_NPI_Aggregate_CY2014.txt')

/Users/zybell/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
NPI.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_cc_depr_percent,beneficiary_cc_diab_percent,beneficiary_cc_hyperl_percent,beneficiary_cc_hypert_percent,beneficiary_cc_ihd_percent,beneficiary_cc_ost_percent,beneficiary_cc_raoa_percent,beneficiary_cc_schiot_percent,beneficiary_cc_strk_percent,Beneficiary_Average_Risk_Score
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,46.0,52.0,70.0,75.0,68.0,14.0,53.0,18.0,20.0,2.2206
1,1003000134,CIBULL,THOMAS,L,M.D.,M,I,2650 RIDGE AVE,EVANSTON HOSPITAL,EVANSTON,...,12.0,20.0,53.0,57.0,28.0,9.0,37.0,1.0,3.0,1.0007
2,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,44.0,48.0,61.0,75.0,41.0,8.0,75.0,9.0,NaN,1.7777
3,1003000407,GIRARDI,DAVID,J,D.O.,M,I,456 MAGEE AVE,NaN,PATTON,...,47.0,40.0,59.0,75.0,62.0,11.0,45.0,19.0,10.0,1.7228
4,1003000423,VELOTTA,JENNIFER,A,M.D.,F,I,11100 EUCLID AVE,NaN,CLEVELAND,...,32.0,23.0,51.0,46.0,19.0,NaN,32.0,NaN,NaN,0.9101


In [4]:
NPI.describe()

,npi,number_of_hcpcs,total_services,total_unique_benes,total_submitted_chrg_amt,total_medicare_allowed_amt,total_medicare_payment_amt,total_medicare_stnd_amt,number_of_drug_hcpcs,total_drug_services,...,beneficiary_cc_depr_percent,beneficiary_cc_diab_percent,beneficiary_cc_hyperl_percent,beneficiary_cc_hypert_percent,beneficiary_cc_ihd_percent,beneficiary_cc_ost_percent,beneficiary_cc_raoa_percent,beneficiary_cc_schiot_percent,beneficiary_cc_strk_percent,Beneficiary_Average_Risk_Score
count,9.866770e+05,986677.000000,9.866770e+05,9.866770e+05,9.866770e+05,9.866770e+05,9.866770e+05,9.866770e+05,883567.000000,8.835670e+05,...,827752.000000,847364.000000,913013.000000,934284.000000,841621.000000,617374.000000,884690.000000,516824.000000,546038.000000,986677.000000
mean,1.499657e+09,32.583957,2.685515e+03,3.684959e+02,3.326049e+05,1.200399e+05,9.209262e+04,9.144818e+04,2.043133,9.705095e+02,...,30.514393,36.445008,57.097108,67.096606,39.645125,9.963272,44.097786,9.281045,8.500361,1.489883
std,2.880157e+08,35.755980,3.637059e+04,2.878940e+03,2.321402e+06,6.110722e+05,5.520058e+05,5.334284e+05,5.593653,1.157377e+04,...,15.100434,12.365744,11.776989,10.043063,15.120318,5.529963,12.954647,12.225311,7.781422,0.720481
min,1.003000e+09,1.000000,1.100000e+01,1.100000e+01,4.500000e+01,2.700000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.271300
25%,1.245577e+09,9.000000,1.930000e+02,6.600000e+01,2.922000e+04,1.378413e+04,1.036583e+04,1.069117e+04,0.000000,0.000000e+00,...,20.000000,28.000000,49.000000,61.000000,28.000000,7.000000,35.000000,2.000000,4.000000,1.005400
50%,1.497966e+09,21.000000,5.590000e+02,1.740000e+02,1.092050e+05,4.287645e+04,3.195630e+04,3.203132e+04,0.000000,0.000000e+00,...,28.000000,35.000000,58.000000,72.000000,37.000000,9.000000,42.000000,6.000000,7.000000,1.294700
75%,1.750301e+09,43.000000,1.632000e+03,3.910000e+02,3.112064e+05,1.163251e+05,8.724571e+04,8.667023e+04,2.000000,3.400000e+01,...,37.000000,43.000000,66.000000,75.000000,50.000000,12.000000,51.000000,11.000000,12.000000,1.796200
max,1.993000e+09,1095.000000,1.210750e+07,1.350844e+06,8.314468e+08,1.701440e+08,1.660135e+08,1.634007e+08,119.000000,3.443171e+06,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,10.972600


In [5]:
NPI.shape

(986677, 70)

In [6]:
# Make a new dataframe with only nppes_entitiy_code equals 'I'
NPI_ind = NPI[(NPI['nppes_entity_code']=='I')]

In [7]:
NPI_ind.shape

(925347, 70)

In [8]:
# Task 1(a):
# Find out the provider who was paid the highest amount by Medicare
task1_table = NPI_ind[['total_medicare_payment_amt', 'nppes_provider_first_name', 'nppes_provider_last_org_name', 'provider_type']]\
.sort_values(['total_medicare_payment_amt'],ascending=[0])
task1_table.head()

,total_medicare_payment_amt,nppes_provider_first_name,nppes_provider_last_org_name,provider_type
31757,14801646.80,ASAD,QAMAR,Cardiology
62717,14721432.49,ANNE,GREIST,Hematology/Oncology
854757,10978357.61,VASSO,GODIALI,Vascular Surgery
544772,10806673.65,DEREK,KUNIMOTO,Ophthalmology
908480,10234125.37,ALEXANDER,EATON,Ophthalmology


Provider ASAD QAMAR, with specialty in Cardiology was paid the hgiest amount by Medicare in 2014, with 14,801,646.80

In [9]:
# Task 1(b):
# make an aggregation table with count and median_payment for each provider type
median_count_specialties = pd.DataFrame(task1_table.groupby('provider_type')\
                                        .agg({'total_medicare_payment_amt':{"amt_median":np.median, "count_providers":np.size}}))  
median_count_specialties.columns = median_count_specialties.columns.droplevel(0)
median_count_specialties = pd.DataFrame(median_count_specialties[(median_count_specialties['count_providers']>=100)]\
                                        .sort_values(['amt_median'],ascending=[0]).to_records())
median_count_specialties.head()

,provider_type,count_providers,amt_median
0,Cardiac Electrophysiology,1512.0,188023.145
1,Ophthalmology,17664.0,185761.335
2,Nephrology,8036.0,184928.690
3,Vascular Surgery,2915.0,165972.400
4,Cardiology,22884.0,161221.905


In [10]:
median_count_specialties.tail()

,provider_type,count_providers,amt_median
69,Maxillofacial Surgery,1008.0,5718.515
70,Oral Surgery (dentists only),1465.0,5580.940
71,Audiologist (billing independently),5851.0,4547.800
72,Registered Dietician/Nutrition Professional,1741.0,4324.710
73,Certified Nurse Midwife,753.0,1692.910


In [11]:
#Look at the rank of "Cardiology"
np.where(median_count_specialties["provider_type"] == 'Cardiology')

(array([4]),)

74 specialties have at least 100 providers
Cardiac Electrophysiology has the highest median payment and the amount is 188023.145
Certified Nurse Midwife has the lowest median payment and the amount is 1692.910
The highest paid provider's specialty 'Cardiology' is ranking 5

In [12]:
# Task 1(c):
# 
# Look at the number of unique specialties in the NPI dataset
t1_unique_provider_type = task1_table.provider_type.unique()  
len(t1_unique_provider_type)  

85

In [13]:
median_unique = median_count_specialties.provider_type.unique()  
len(median_unique)

74

In [14]:
# Make a new table called "task1_over100_table" which only contains 'provider_type' with >=100 providers
task1_over100_table = task1_table[task1_table['provider_type'].\
                                  isin(median_count_specialties['provider_type'])]

In [15]:
task1_over100_table.shape

(925054, 4)

In [16]:
# Only keep the 10 highest paid providers in each specialty
task1_over100_top10 = task1_over100_table.groupby('provider_type').head(10).reset_index(drop=True)

In [17]:
task1_over100_top10.head()

,total_medicare_payment_amt,nppes_provider_first_name,nppes_provider_last_org_name,provider_type
0,14801646.80,ASAD,QAMAR,Cardiology
1,14721432.49,ANNE,GREIST,Hematology/Oncology
2,10978357.61,VASSO,GODIALI,Vascular Surgery
3,10806673.65,DEREK,KUNIMOTO,Ophthalmology
4,10234125.37,ALEXANDER,EATON,Ophthalmology


In [18]:
median_count_2 = pd.DataFrame(task1_over100_top10.groupby('provider_type')\
                                        .agg({'total_medicare_payment_amt':{"amt_median":np.median, "count_providers":np.size}}))  

median_count_2.columns = median_count_2.columns.droplevel(0)

median_count_2 = pd.DataFrame(median_count_2.sort_values(['amt_median'],ascending=[0]).to_records())
median_count_2.head(10)

,provider_type,count_providers,amt_median
0,Ophthalmology,10.0,7531933.360
1,Hematology/Oncology,10.0,7222146.850
2,Medical Oncology,10.0,4945725.705
3,Radiation Oncology,10.0,4421019.590
4,Rheumatology,10.0,4416689.330
5,Cardiology,10.0,4279912.970
6,Vascular Surgery,10.0,3436768.035
7,Pathology,10.0,3415477.020
8,Internal Medicine,10.0,3351461.960
9,Interventional Radiology,10.0,3202738.175


In [19]:
median_count_2.tail()

,provider_type,count_providers,amt_median
69,Maxillofacial Surgery,10.0,147538.400
70,Audiologist (billing independently),10.0,136383.315
71,Registered Dietician/Nutrition Professional,10.0,120012.190
72,Anesthesiologist Assistants,10.0,53349.100
73,Certified Nurse Midwife,10.0,24631.045


In [20]:
#Look at the rank of "Cardiology" in the 10 highest providers per provider_type
np.where(median_count_2["provider_type"] == 'Cardiology')

(array([5]),)

Ophthalmology has the highest median payment among its 10 highest paid provider with 7531933.360
    Certified Nurse Midwife has the lowest median payment among its 10 highest paid provider with 24631.045
    The highest paid provider's specialty 'Cardiology' is ranking 6